In [1]:
import time
import numpy as np
from src.data_loader import load_data
from src.feature_engineering import average_daily
from src.model.train_model import get_X_y, train_and_evaluate_models
import os
from objective_function import objective_et, objective_lasso
from src.Optimiser.KOA.koa_optimizer import koa_optimizer
from src.Optimiser.COA.coa_optimizer import coa_optimizer
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from analysis.wilcoxon_test import perform_wilcoxon_test

DATA_PATH = os.path.join("data", "Data.xlsx")


In [3]:
print("🔹 Loading raw data...")
df = load_data(DATA_PATH)
print(f"✅ Raw data shape: {df.shape}")

🔹 Loading raw data...
✅ Raw data shape: (50000, 12)


In [5]:
print("🔹 Aggregating daily averages (96 samples per day)...")
df_avg = average_daily(df, samples_per_day=96)
print(f"✅ Aggregated data shape: {df_avg.shape}")
print("✅ Sample preview:")
print(df_avg.head())

🔹 Aggregating daily averages (96 samples per day)...
✅ Aggregated data shape: (520, 12)
✅ Sample preview:
   Voltage (V)  Current (A)  Power Consumption (kW)  Temperature (Â°C)  \
0   229.442065    23.274581                5.338170          24.458245   
1   230.252406    27.590016                6.355897          25.281132   
2   230.139098    26.616651                6.113400          25.286834   
3   230.450687    28.109668                6.483036          26.025139   
4   230.131332    26.532084                6.102769          24.729869   

   Humidity (%)  Reactive Power (kVAR)  Power Factor  Solar Power (kW)  \
0     49.931362               1.335934      0.904576         24.128362   
1     47.592609               1.547024      0.902831         25.205873   
2     46.656764               1.560915      0.898813         22.725012   
3     51.258872               1.580661      0.905492         24.628754   
4     49.909403               1.526028      0.902028         23.925562   

   W

In [6]:
print("🔹 Preparing features and target...")
X, y = get_X_y(df_avg)
print(f"✅ X shape: {X.shape}, y shape: {y.shape}")


🔹 Preparing features and target...
✅ X shape: (520, 11), y shape: (520,)


In [ ]:
print("🔹 Training baseline models with 5-Fold CV...")
start_time = time.time()
etr_scores, lasso_scores = train_and_evaluate_models(X, y)
elapsed_time = time.time() - start_time
print(f"\n⏱️ Total training and evaluation time: {elapsed_time:.2f} seconds")

🔹 Training baseline models with 5-Fold CV...

🔁 Fold 1 ------------------
  ETR → R2: 0.9897, RMSE: 0.0012, MAE: 0.0213, MAPE: 0.35%, MARD: 0.17%
  Lasso → R2: -0.0207, RMSE: 0.1208, MAE: 0.2784, MAPE: 4.51%, MARD: 2.22%

🔁 Fold 2 ------------------
  ETR → R2: 0.9932, RMSE: 0.0006, MAE: 0.0200, MAPE: 0.32%, MARD: 0.16%
  Lasso → R2: -0.0005, RMSE: 0.0904, MAE: 0.2362, MAPE: 3.75%, MARD: 1.87%

🔁 Fold 3 ------------------
  ETR → R2: 0.9909, RMSE: 0.0011, MAE: 0.0209, MAPE: 0.32%, MARD: 0.16%
  Lasso → R2: -0.0524, RMSE: 0.1227, MAE: 0.2708, MAPE: 4.19%, MARD: 2.12%

🔁 Fold 4 ------------------
  ETR → R2: 0.9905, RMSE: 0.0006, MAE: 0.0200, MAPE: 0.32%, MARD: 0.16%
  Lasso → R2: -0.0130, RMSE: 0.0656, MAE: 0.2026, MAPE: 3.26%, MARD: 1.61%

🔁 Fold 5 ------------------
  ETR → R2: 0.9953, RMSE: 0.0004, MAE: 0.0166, MAPE: 0.26%, MARD: 0.13%
  Lasso → R2: -0.0004, RMSE: 0.0916, MAE: 0.2385, MAPE: 3.77%, MARD: 1.88%

⏱️ Total training and evaluation time: 1.42 seconds


In [16]:
# show the scores
print("\n🔹 Baseline Model Scores:")
print(f"Extra Trees Regressor MAE: {np.array(etr_scores)}")
print(f"Lasso Regression MAE: {np.array(lasso_scores)}")


🔹 Baseline Model Scores:
Extra Trees Regressor MAE: [{'R2': 0.9896722490011709, 'RMSE': 0.0012223647690544433, 'MAE': 0.021315656403234465, 'MAPE': np.float64(0.34904662106832834), 'MARD': np.float64(0.1738597306469053)}
 {'R2': 0.993186161656994, 'RMSE': 0.0006156158103078235, 'MAE': 0.02001144974400134, 'MAPE': np.float64(0.31645122020371796), 'MARD': np.float64(0.15833867938927135)}
 {'R2': 0.9908695252796009, 'RMSE': 0.0010643204704476321, 'MAE': 0.02092063174825827, 'MAPE': np.float64(0.32271027665259405), 'MARD': np.float64(0.16162781291648215)}
 {'R2': 0.9905351690993972, 'RMSE': 0.0006132430981234915, 'MAE': 0.02003457982731975, 'MAPE': np.float64(0.317287451396465), 'MARD': np.float64(0.15861377655943645)}
 {'R2': 0.9952712193436908, 'RMSE': 0.0004329537471037881, 'MAE': 0.01655667005711755, 'MAPE': np.float64(0.2623814681462597), 'MARD': np.float64(0.1311971543157486)}]
Lasso Regression MAE: [{'R2': -0.020739805658197108, 'RMSE': 0.12081201192297576, 'MAE': 0.278362192178098

In [20]:
def summarize_metrics(metrics_list):
  return {key: np.mean([m[key] for m in metrics_list]) for key in metrics_list[0]}

avg_etr = summarize_metrics(etr_scores)
avg_lasso = summarize_metrics(lasso_scores)

print("\n🔹 Average Baseline Model Scores:")
print(f"Extra Trees Regressor MAE: {avg_etr['MAE']:.4f}, R2: {avg_etr['R2']:.4f}")
print(f"Lasso Regression MAE: {avg_lasso['MAE']:.4f}, R2: {avg_lasso['R2']:.4f}")


🔹 Average Baseline Model Scores:
Extra Trees Regressor MAE: 0.0198, R2: 0.9919
Lasso Regression MAE: 0.2453, R2: -0.0174


In [ ]:



def main():
    print("🔹 Extra Trees Regressor:")
    for k, v in avg_etr.items():
        print(f"  {k}: {v:.4f}")

    print("\n🔹 Lasso Regression:")
    for k, v in avg_lasso.items():
        print(f"  {k}: {v:.4f}")

    # Parameter bounds for Extra Trees
    lb_et = [10, 5, 2, 1]          # n_estimators, max_depth, min_samples_split, min_samples_leaf
    ub_et = [300, 50, 10, 10]
    dim_et = 4

    # Parameter bounds for Lasso
    lb_lasso = [0.0001, 100]       # alpha, max_iter
    ub_lasso = [1.0, 1000]
    dim_lasso = 2

    n_agents = 3
    max_iter = 5

    # Optimize Extra Trees with KOA
    print("\n🔹 Optimizing Extra Trees Regressor with KOA...")
    best_params_koa_et, best_mae_koa_et = koa_optimizer(objective_et, lb_et, ub_et, dim_et, n_agents, max_iter, X, y)
    print(f"ETR + KOA Best params: {best_params_koa_et}")
    print(f"ETR + KOA Best MAE: {best_mae_koa_et:.5f}")

    # Optimize Extra Trees with COA
    print("\n🔹 Optimizing Extra Trees Regressor with COA...")
    best_params_coa_et, best_mae_coa_et = coa_optimizer(objective_et, lb_et, ub_et, dim_et, n_agents, max_iter, X, y)
    print(f"ETR + COA Best params: {best_params_coa_et}")
    print(f"ETR + COA Best MAE: {best_mae_coa_et:.5f}")

    # Optimize Lasso with KOA
    print("\n🔹 Optimizing Lasso Regression with KOA...")
    best_params_koa_lasso, best_mae_koa_lasso = koa_optimizer(objective_lasso, lb_lasso, ub_lasso, dim_lasso, n_agents, max_iter, X, y)
    print(f"Lasso + KOA Best params: {best_params_koa_lasso}")
    print(f"Lasso + KOA Best MAE: {best_mae_koa_lasso:.5f}")

    # Optimize Lasso with COA
    print("\n🔹 Optimizing Lasso Regression with COA...")
    best_params_coa_lasso, best_mae_coa_lasso = coa_optimizer(objective_lasso, lb_lasso, ub_lasso, dim_lasso, n_agents, max_iter, X, y)
    print(f"Lasso + COA Best params: {best_params_coa_lasso}")
    print(f"Lasso + COA Best MAE: {best_mae_coa_lasso:.5f}")

    # Wilcoxon Test on baseline models
    mae_etr = [score['MAE'] for score in etr_scores]
    mae_lr = [score['MAE'] for score in lasso_scores]
    perform_wilcoxon_test(mae_etr, mae_lr, model_name_1="ETR", model_name_2="LR")

if __name__ == "__main__":
    main()
